# DPO Training -> ONNX 변환

1. DPO 학습 실행
2. 학습된 모델을 ONNX로 변환

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

Fri Nov  7 05:59:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             50W /  400W |   32489MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# 필수 라이브러리 설치
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets accelerate peft trl
!pip install -q bitsandbytes scipy
!pip install -q wandb
!pip install -q deepspeed

# ONNX 변환용 라이브러리
!pip install -q optimum[exporters]
!pip install -q onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mymymy` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.ca

## DPO 학습 실행


In [ ]:
!python run_dpo_colab_fixed.py \
  --model_name_or_path "meta-llama/Llama-3.1-8B-Instruct" \
  --dataset_name "SJ-Donald/orca-dpo-pairs-ko" \
  --output_dir "./dpo_output" \
  --num_train_epochs 1 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1 \
  --gradient_accumulation_steps 4 \
  --learning_rate 5e-5 \
  --beta 0.1 \
  --bf16 True \
  --logging_steps 10 \
  --eval_strategy steps \
  --eval_steps 50 \
  --save_strategy steps \
  --save_steps 50 \
  --save_total_limit 1 \
  --gradient_checkpointing True \
  --lora_r 32 \
  --lora_alpha 64

2025-11-07 05:14:08.435134: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-07 05:14:08.454921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762492448.477376   11943 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762492448.484023   11943 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762492448.500906   11943 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
# 모든 관련 라이브러리 최신 버전으로 업데이트
!pip install -U transformers optimum
!pip install -U accelerate peft onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 115.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum-onnx 0.0.3 requires transformers<4.56.0,>=4.36.0, but you have transformers 4.57.1 which is incompatible.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

## 학습된 모델 병합 (LoRA → Full Model)

ONNX 변환 전에 LoRA 어댑터를 베이스 모델과 병합합니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import os

base_model_name = "meta-llama/Llama-3.1-8B-Instruct"
lora_path = "./dpo_output"  # LoRA로 학습된 adapter weight가 저장되어있는곳
save_dir = "./dpo_merged_model2"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Base 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# LoRA 병합
peft_config = PeftConfig.from_pretrained(lora_path)  # LoRA config 로드

model = PeftModel.from_pretrained(base_model, lora_path)  # Base 모델에 LoRA adapter를 넣고 로드



"""
LoRA 가중치를 Base 모델 가중치에 덧셈 병합(Merge)
LoRA adapter 레이어 제거 (unload)
-> 일반 모델처럼 완전한 full-weight 모델
"""



model = model.merge_and_unload()
model.eval()

# 저장
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("LoRA merge 완료 → saved in:", save_dir)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='meta-llama/Llama-3.1-8B-Instruct', revision=None, inference_mode=True, r=32, target_modules={'gate_proj', 'q_proj', 'down_proj', 'v_proj', 'o_proj', 'k_proj', 'up_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)
LoRA merge 완료 → saved in: ./dpo_merged_model2


##  ONNX 변환

LoRA 병합된 모델을 Hugging Face Optimum을 이용해 ONNX 형식으로 내보내고
해당 ONNX 모델을 저장  

1. **ORTModelForCausalLM**  
ONNX Runtime 백엔드에서 causal LM 모델을 실행하기 위한 Optimum 클래스  
PyTorch 모델을 ONNX로 자동 변환(export=True) 하는 기능 제공 -> 내부적으로 torch.onnx.export() 실행

### Optimum
Hugging Face Optimum: HuggingFace Transformers 모델을 최적화하고 추론할 수 있도록 도와주는 프레임워크

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

onnx_dir = "./dpo_onnx_model"



"""
모델을 onnx 포맷으로 변환한 후 다시 ORTModelForCaualLM 객체로 wrapped
=> ONNX Runtime 을 추론 백엔드로 사용하면서
   HuggingFace Transformers API처럼 모델을 사용할 수 있도록

그런데 TensorRT까지 적용하려면 ORTModel wrapper가 필요없고 그냥 onnx 변환모델만 있으면 되니까
순정 torch.onnx.export() 결과만 사용
"""


model_ort = ORTModelForCausalLM.from_pretrained(
    save_dir,      # merged model path
    export=True    # PyTorch → ONNX 변환
)


tokenizer = AutoTokenizer.from_pretrained(save_dir)


# 저장
model_ort.save_pretrained(onnx_dir)
tokenizer.save_pretrained(onnx_dir)


print("ONNX 변환 완료 → saved in:", onnx_dir)

### 한 15분정도 걸렸습니다

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/cache_utils.py:94: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  self.keys = torch.tensor([], dtype=self.dtype, device=self.device)
/usr/local/lib/python3.12/dist-packages/transformers/cache_utils.py:95: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  self.values = torch.tensor([], dtype=self.dtype, device=self.device)
/usr/local/lib/python3.12/dist-packages/transformers/cache_utils.py:132: TracerWarning: Converting a tensor to

ONNX 변환 완료 → saved in: ./dpo_onnx_model


In [ ]:
"""
TensorRT, vLLM 등
"""

## ONNX 모델 테스트

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

# ONNX 모델 로드
print("ONNX 모델 테스트...")
onnx_model = ORTModelForCausalLM.from_pretrained("./dpo_onnx_model")
tokenizer = AutoTokenizer.from_pretrained("./dpo_onnx_model")

# 테스트 입력
test_prompt = "멋있는 팀 이름을 추천해줘. AI 관련 일을 하는 팀이야."
inputs = tokenizer(test_prompt, return_tensors="pt")

# 생성
outputs = onnx_model.generate(**inputs, max_new_tokens=100)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"\n입력: {test_prompt}")
print(f"출력: {generated_text}")

##  모델 크기 비교

In [ ]:
import os

def get_dir_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total += os.path.getsize(filepath)
    return total / (1024**3)  # GB로 변환

print("\n=== 모델 크기 비교 ===")
print(f"LoRA 어댑터 모델: {get_dir_size('./dpo_output'):.2f} GB")
print(f"병합된 PyTorch 모델: {get_dir_size('./dpo_merged_model'):.2f} GB")
print(f"ONNX 모델: {get_dir_size('./dpo_onnx_model'):.2f} GB")